# StockCharts

This notebook is meant to collect data from stockcharts.com from Industry and Sector performances.  

To Do:  
* Build Sector and Industry Dictionaries  
* Build Beautiful Soup data pull  
* Build dataframe  
* Load to PostgreSQL using SQLAlchemy  
* Calculate dollar volumes by cross-referencing Finviz or yfinance Shares Oustanding data

In [5]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup as soup

# Instantiate an Options object and add the --headless argument
# This runs the browser in the background
opts = Options()
opts.add_argument(" --headless")

# Set the location of the webdriver
chrome_driver='C:\\Users\\srobi\\Projects\\chromedriver.exe'

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts,executable_path=chrome_driver)

# To scrape a url rather than a local file use this
my_url = 'https://stockcharts.com/freecharts/industrysummary.html#&O=6'
driver.get(my_url)

# Put the page source into a variable and create a BS object from it, parse HTML
page_html = driver.page_source
page_soup = soup(page_html, "html.parser")

# Close the browser
driver.quit()

In [6]:
# Tests to see if html has been parsed properly
print('\nTesting html parsing...')
print(page_soup.title.get_text())
print(page_soup.h1.text) # shows header information
print(page_soup.p.text) # shows paragraph information
print(page_soup.body.span.text)
print('Testing completed.\n')


Testing html parsing...
Industry Summary | StockCharts.com
Industry Summary
Simple Moving Averages: 20-period / 50-period / 200-period
Welcome
Testing completed.



In [13]:
# Grabs each product
index_symbols = page_soup.findAll("span",{"class":"index-symbol"})
print(f'There are {len(index_symbols)} total.') # find total number of containers
index_symbol = index_symbols[0].text # look at the first container
print(f'Here is the first title: \n{index_symbol}\n')

# paragraphs = page_soup.findAll("div",{"class":"article_teaser_body"})
# print(f'There are {len(paragraphs)} paragraphs total.') # find total number of containers
# news_p = paragraphs[0].text # look at the first container
# print(f'Here is the first paragraph: \n{news_p}\n')

There are 104 total.
Here is the first title: 
$DJUSAV



In [14]:
# Grabs each product
index_names = page_soup.findAll("span",{"class":"index-name"})
print(f'There are {len(index_names)} titles total.') # find total number of containers
index_name = index_names[0].text # look at the first container
print(f'Here is the first title: \n{index_name}\n')

There are 104 titles total.
Here is the first title: 
Media Agencies



In [23]:
index_symbols_list = []
index_names_list = []

for x in range(len(index_names)):
    index_symbols_list.append(index_symbols[x].text)
    index_names_list.append(index_names[x].text)

index_dict = {"index_symbol":index_symbols_list,"index_name":index_names_list}

In [25]:
import pandas as pd

df = pd.DataFrame(index_dict)
df

,index_symbol,index_name
0,$DJUSAV,Media Agencies
1,$DJUSBC,Broadcasting & Entertainment
2,$DJUSFC,Fixed Line Telecommunications
3,$DJUSNS,Internet
4,$DJUSPB,Publishing
...,...,...
99,$DWCREE,Renewable Energy Equipment
100,$DJUSGU,Gas Distribution
101,$DJUSMU,Multiutilities
102,$DJUSVE,Conventional Electricity


In [20]:
index_dict

[{'index_symbol': '$DJUSAV', 'index_name': 'Media Agencies'},
 {'index_symbol': '$DJUSBC', 'index_name': 'Broadcasting & Entertainment'},
 {'index_symbol': '$DJUSFC', 'index_name': 'Fixed Line Telecommunications'},
 {'index_symbol': '$DJUSNS', 'index_name': 'Internet'},
 {'index_symbol': '$DJUSPB', 'index_name': 'Publishing'},
 {'index_symbol': '$DJUSWC', 'index_name': 'Mobile Telecommunications'},
 {'index_symbol': '$DJUSAT', 'index_name': 'Auto Parts'},
 {'index_symbol': '$DJUSAU', 'index_name': 'Automobiles'},
 {'index_symbol': '$DJUSBE',
  'index_name': 'Business Training & Employment Agencies'},
 {'index_symbol': '$DJUSCA', 'index_name': 'Gambling'},
 {'index_symbol': '$DJUSCF', 'index_name': 'Clothing & Accessories'},
 {'index_symbol': '$DJUSCS', 'index_name': 'Specialized Consumer Services'},
 {'index_symbol': '$DJUSFH', 'index_name': 'Furnishings'},
 {'index_symbol': '$DJUSFT', 'index_name': 'Footwear'},
 {'index_symbol': '$DJUSHB', 'index_name': 'Home Construction'},
 {'index_

In [15]:
for name in index_names:
    print(name.text)

Media Agencies
Broadcasting & Entertainment
Fixed Line Telecommunications
Internet
Publishing
Mobile Telecommunications
Auto Parts
Automobiles
Business Training & Employment Agencies
Gambling
Clothing & Accessories
Specialized Consumer Services
Furnishings
Footwear
Home Construction
Durable Household Products
Home Improvement Retailers
Hotels
Apparel Retailers
Broadline Retailers
Recreational Products
Recreational Services
Specialty Retailers
Restaurants & Bars
Travel & Tourism
Toys
Tires
Personal Products
Brewers
Food Retailers & Wholesalers
Food Products
Nondurable Household Products
Drug Retailers
Soft Drinks
Tobacco
Distillers & Vintners
Oil Equipment & Services
Integrated Oil & Gas
Exploration & Production
Pipelines
Coal
Asset Managers
Banks
Financial Administration
Insurance Brokers
Full Line Insurance
Life Insurance
Property & Casualty Insurance
Reinsurance
Mortgage Finance
Investment Services
Consumer Finance
Specialty Finance
Medical Equipment
Biotechnology
Health Care Provide

In [ ]:
url = 'https://stockcharts.com/freecharts/sectorsummary.html?&G='

sector_list = {"communication_services":['media_agencies','broadcasting_and_entertainment','fixed_line_telecommunications','internet','publishing','mobile_telecommunications']}

industry_list = {"media_agencies":"DJUSAV",
                 "broadcasting_and_entertainment":"DJUSBC",
                "fixed_line_telecommunications":"DJUSFC",
                "internet":"DJUSNS",
                "publishing":"DJUSPB",
                "mobile_telecommunications":"DJUSWC",
                "auto_parts":"DJUSAT",
                "automobiles":"DJUSAU",
                "business_training_and_employment_agencies":"DJUSBE",
                "gambling":"DJUSBE",
                "clothing_and_accessories":"DJUSBE",
                "specialized_consumer_services":"DJUSBE",
                "furnishings":"DJUSBE",
                "footwear":"DJUSBE",
                "home_construction":"DJUSBE",
                "durable_household_products":"DJUSBE",
                "home_improvement_retailers":"DJUSBE",
                "hotels":"DJUSBE",
                "apparel_retailers":"DJUSBE",
                "broadline_retailers":"DJUSBE",
                "recreational_products":"DJUSBE",
                "recreational_services":"DJUSBE",
                "specialty_retailers":"DJUSBE",
                "restaurants_and_bars":"DJUSBE",
                "travel_and_tourism":"DJUSBE",
                "toys":"DJUSBE",
                "tires":"DJUSBE",
                "personal_products":"DJUSBE",
                "brewers":"DJUSBE",
                "food_retailers_and_wholesalers":"DJUSBE",
                "food_products":"DJUSBE",
                "nondurable_household_products":"DJUSBE",
                "drug_retailers":"DJUSBE",
                "soft_drinks":"DJUSBE",
                "tobacco":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "oil_equipment_and_services":"DJUSBE",
                "integrated_oil_and_gas":"DJUSBE",
                "exploration_and_production":"DJUSBE",
                "pipelines":"DJUSBE",
                "coal":"DJUSBE",
                "asset_managers":"DJUSBE",
                "banks":"DJUSBE",
                "financial_administration":"DJUSBE",
                "insurance_brokers":"DJUSBE",
                "full_line_insurance":"DJUSBE",
                "life_insurance":"DJUSBE",
                "property_and_casualty_insurance":"DJUSBE",
                "reinsurance":"DJUSBE",
                "mortgage_finance":"DJUSBE",
                "investment_services":"DJUSBE",
                "consumer_finance":"DJUSBE",
                "specialty_finance":"DJUSBE",
                "medical_equipment":"DJUSBE",
                "biotechnology":"DJUSBE",
                "health_care_providers":"DJUSBE",
                "medical_supplies":"DJUSBE",
                "pharmaceuticals":"DJUSBE",
                "delivery_services":"DJUSBE",
                "airlines":"DJUSBE",
                "aerospace":"DJUSBE",
                "building_materials_and_fixtures":"DJUSBE",
                "defense":"DJUSBE",
                "industrial_suppliers":"DJUSBE",
                "industrial_machinery":"DJUSBE",
                "commercial_vehicles_and_trucks":"DJUSBE",
                "heavy_construction":"DJUSBE",
                "diversified_industrials":"DJUSBE",
                "business_support_services":"DJUSBE",
                "marine_transportation":"DJUSBE",
                "aluminum":"DJUSBE",
                "commodity_chemicals":"DJUSBE",
                "containers_and_packaging":"DJUSBE",
                "specialty_chemicals":"DJUSBE",
                "mining":"DJUSBE",
                "nonferrous_metals":"DJUSBE",
                "gold_mining":"DJUSBE",
                "steel":"DJUSBE",
                "paper":"DJUSBE",
                "diversified_reits":"DJUSBE",
                "real_estate_holding_and_development":"DJUSBE",
                "real_estate_services":"DJUSBE",
                "hotel_and_lodging_reits":"DJUSBE",
                "industrial_and_office_reits":"DJUSBE",
                "mortgage_reits":"DJUSBE",
                "retail_reits":"DJUSBE",
                "residential_reits":"DJUSBE",
                "specialty_reits":"DJUSBE",
                "electronic_equipment":"DJUSBE",
                "computer_hardware":"DJUSBE",
                "telecommunications_equipment":"DJUSBE",
                "computer_services":"DJUSBE",
                "electrical_components_and_equipment":"DJUSBE",
                "semiconductors":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",
                "distillers_and_vintners":"DJUSBE",}
industry_list = ['DJUSAV','DJUSBC','DJUSFC','DJUSNC','DJUSBC','DJUSBC','DJUSBC']